#### extensible software in julia
Today we are going to follow some basic design parameters in order to create extensible software for julia.
- everything starts with our type hierarchy, which is incredibly important:

In [1]:
abstract type AbstractParametricPair end

mutable struct ParametricPair <: AbstractParametricPair
    first::Any
    second::Any
end

In [2]:
import Base: getindex, setindex!, string

In [3]:
getindex(pp::AbstractParametricPair, n::Int64) = begin
    if n > 2 || n < 1
        throw(BoundsError(pp,[n]))
    elseif n == 2
        return(pp.second)
    elseif n == 1
        return(pp.first)
    end
end

getindex (generic function with 227 methods)

In [4]:
function string(p::AbstractParametricPair)
    "$(string(p[1]))$(string(p[2]))"
end

string (generic function with 22 methods)

In [5]:
parpair = ParametricPair("hello ", "world")

ParametricPair("hello ", "world")

In [6]:
parpair[1]

"hello "

In [7]:
parpair[-1]

LoadError: BoundsError: attempt to access ParametricPair at index [-1]

In [8]:
parpair[2]

"world"

In [9]:
parpair[3]

LoadError: BoundsError: attempt to access ParametricPair at index [3]

In [10]:
string(parpair)

"hello world"

In [11]:
Pair{<:Any, <:Any} == Pair

true

In [12]:
#==
abstract type AbstractParametricPair end

mutable struct ParametricPair{FIRST, SECOND, PT} <: AbstractParametricPair
    first::FIRST
    second::SECOND
end

LoadError: syntax: incomplete: unterminated multi-line comment #= ... =#

In [13]:
module ParametricPairs
import Base: getindex, setindex!, string

abstract type AbstractParametricPair end

mutable struct ParametricPair{FIRST, SECOND, PT} <: AbstractParametricPair
    first::FIRST
    second::SECOND
    ParametricPair(first::Any, second::Any, type::Any) = begin
        new{typeof(first), typeof(second), type}(first, second)
    end
end

getindex(pp::AbstractParametricPair, n::Int64) = begin
    if n > 2 || n < 1
        throw(BoundsError(pp,[n]))
    elseif n == 2
        return(pp.second)
    elseif n == 1
        return(pp.first)
    end
end

function string(pps::AbstractParametricPair ...)
    join(["$(string(p[1]))$(string(p[2]))" for p in pps])::String
end

pp(a::Any, a2::Any, par::Any = :standard) = begin
    ParametricPair(a, a2, par)
end

export pp
end # parametric pairs :)

Main.ParametricPairs

In [14]:
using Main.ParametricPairs

In [15]:
mypair = pp("firstelement", "secondelement")

Main.ParametricPairs.ParametricPair{String, String, :standard}("firstelement", "secondelement")

In [16]:
mypps = [pp(e, "number $e") for e in 1:500000]

500000-element Vector{Main.ParametricPairs.ParametricPair{Int64, String, :standard}}:
 Main.ParametricPairs.ParametricPair{Int64, String, :standard}(1, "number 1")
 Main.ParametricPairs.ParametricPair{Int64, String, :standard}(2, "number 2")
 Main.ParametricPairs.ParametricPair{Int64, String, :standard}(3, "number 3")
 Main.ParametricPairs.ParametricPair{Int64, String, :standard}(4, "number 4")
 Main.ParametricPairs.ParametricPair{Int64, String, :standard}(5, "number 5")
 Main.ParametricPairs.ParametricPair{Int64, String, :standard}(6, "number 6")
 Main.ParametricPairs.ParametricPair{Int64, String, :standard}(7, "number 7")
 Main.ParametricPairs.ParametricPair{Int64, String, :standard}(8, "number 8")
 Main.ParametricPairs.ParametricPair{Int64, String, :standard}(9, "number 9")
 Main.ParametricPairs.ParametricPair{Int64, String, :standard}(10, "number 10")
 Main.ParametricPairs.ParametricPair{Int64, String, :standard}(11, "number 11")
 Main.ParametricPairs.ParametricPair{Int64, String, 

In [17]:
using BenchmarkTools

In [18]:
@benchmark [(pp[1], pp[2]) for pp in mypps]

BenchmarkTools.Trial: 1761 samples with 1 evaluation.
 Range (min … max):  2.379 ms …   8.468 ms  ┊ GC (min … max): 0.00% … 30.28%
 Time  (median):     2.622 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.825 ms ± 492.503 μs  ┊ GC (mean ± σ):  2.89% ±  5.65%

   ▄▅▆█▃                                                       
  ▆██████▆▅▅▄▄▄▅▆▆▅▅▅▄▄▄▃▄▃▄▂▃▃▃▃▃▃▃▃▂▂▂▂▂▃▂▂▂▂▂▂▂▁▂▂▂▂▂▂▁▂▁▂ ▃
  2.38 ms         Histogram: frequency by time        4.53 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 3.

In [19]:
function string(ppairs::ParametricPairs.ParametricPair{String, <:Any, :label} ...)
    println(join(["$(ppair[1]): $(ppair[2])" for ppair in ppairs], "\n"))
end

string (generic function with 24 methods)

In [20]:
string(pp("A is", 5, :label), pp("B is", 6, :label))

A is: 5
B is: 6


In [21]:
abstract type ParameterEntry end
abstract type Repeat{N} <: ParameterEntry end

In [30]:
function string(ppa::ParametricPairs.ParametricPair{String, <:Any, <:ParameterEntry})
    repeatlen = typeof(ppa).parameters[3].parameters[1]
    join(["$(ppa.first): $(ppa.second)" for ppair in 1:repeatlen], "\n")
end

string (generic function with 26 methods)

In [31]:
string(pp("hello", "friends!", Repeat{15}))

"hello: friends!\nhello: friends!\nhello: friends!\nhello: friends!\nhello: friends!\nhello: friends!\nhello: friends!\nhello: friends!\nhello: friends!\nhello: friends!\nhello: friends!\nhello: friends!\nhello: friends!\nhello: friends!\nhello: friends!"